In [ ]:
import urllib.request 
import numpy as np
from PIL import Image
from paddleocr import PaddleOCR


def img_to_text():
    image_url='https://groups.google.com/group/jzebra-users/attach/d16dbba8a612edfa/Bill%20Image_Receipt.png?part=0.1'
    local_image_id='bill_image_receipt.png'
    urllib.request.urlretrieve(image_url,local_image_id) 
    receipt_image = Image.open(local_image_id)
    receipt_image_array = np.array(receipt_image.convert('RGB'))
    
    ocr = PaddleOCR(lang="en",
                ocr_version="PP-OCRv4")
    

    result = ocr.predict(receipt_image_array)
    result = result[0]
    txts = result['rec_texts']
    
    return txts


import torch 
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, BitsAndBytesConfig



def model():

    bnb_config = BitsAndBytesConfig(
        llm_int8_enable_fp32_cpu_offload=True,
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    device_map = {
        "transformer.word_embeddings": 0,
        "transformer.word_embeddings_layernorm": 0,
        "lm_head": 0,
        "transformer.h": 0,
        "transformer.ln_f": 0,
        "model.embed_tokens": 0,
        "model.layers":0,
        "model.norm":0    
    }

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model_id="mychen76/mistral7b_ocr_to_json_v1"

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        quantization_config=bnb_config,
        # device_map=device_map
    )

    tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          trust_remote_code=True)


    return model, tokenizer





if __name__ == "__main__":

    receipt_texts = img_to_text()

    prompt=f"""### Instruction:
    You are POS receipt data expert, parse, detect, recognize and convert following receipt OCR image result into structure receipt data object. 
    Don't make up value not in the Input. Output must be a well-formed JSON object.```json

    ### Input:
    {receipt_texts}

    ### Output:
    """

    load_model, tokenizer = model()

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    with torch.inference_mode():
        inputs = tokenizer(prompt,
                        return_tensors="pt",
                        truncation=True).to(device)
        
       
        
        outputs = load_model.generate(**inputs, max_new_tokens=512)
        outputs = tokenizer.batch_decode(outputs)
        print(outputs)

        output_file = "receipt_output.txt"

        # Open a new file in write mode
        with open(output_file, "w") as f:
            # Write the raw output text to the file
            # The [0] is because tokenizer.batch_decode returns a list
            # of one string, so we want to get the first and only element.
            f.write(outputs[0])



output: 

```
    <s> ### Instruction:
    You are POS receipt data expert, parse, detect, recognize and convert following receipt OCR image result into structure receipt data object. 
    Don't make up value not in the Input. Output must be a well-formed JSON object.```json

    ### Input:
    ['The Lone Pine', '43 Manchester Road', '12480 Brisbane', 'Australia', '617-3236-6207', 'Invoice 08000008', '09/04/08', 'Table', '25', '12:45', '2 Carlsberg Bottle', '16.00', '3 Heineken Draft Standard', '24.60', '1 Heineken Draft Half Liter', '15.20', '2 Carlsberg Bucket (5 bottles)', '80.00', '4 Grilled Chicken Breast', '74.00', '3 Sirloin Steak', '96.00', '1 Coke', '3.50', '5 Ice Cream', '18.00', 'Subtotal', '327.30', 'Sales/Gov Tax - 5%', '16.36', 'Service Charge - 10%', '32.73', 'GRAND TOTAL', '376.40', 'Thank you and', 'Cash', '400.00', 'see you again!', 'Change', '23.60', 'John', 'Bring this bill back within the next 10 days', "and get 15% discount on that day's food bill."]

    ### Output:
     {
        "store": "The Lone Pine",
        "address": "43 Manchester Road, 12480 Brisbane, Australia",
        "phone": "617-3236-6207",
        "invoice": "Invoice 08000008",
        "date": "09/04/08",
        "table": "Table",
        "people": "25",
        "time": "12:45",
        "items": [
            {
                "name": "Carlsberg Bottle",
                "price": "16.00",
                "quantity": "2"
            },
            {
                "name": "Heineken Draft Standard",
                "price": "24.60",
                "quantity": "3"
            },
            {
                "name": "Heineken Draft Half Liter",
                "price": "15.20",
                "quantity": "1"
            },
            {
                "name": "Carlsberg Bucket (5 bottles)",
                "price": "80.00",
                "quantity": "2"
            },
            {
                "name": "Grilled Chicken Breast",
                "price": "74.00",
                "quantity": "4"
            },
            {
                "name": "Sirloin Steak",
                "price": "96.00",
                "quantity": "3"
            },
            {
                "name": "Coke",
                "price": "3.50",
                "quantity": "1"
            },
            {
                "name": "Ice Cream",
                "price": "18.00",
                "quantity": "5"
            }
        ],
        "subtotal": "327.30",
        "tax": "16.36",
        "service": "32.73",
        "total
```

In [ ]:
import urllib.request 
import numpy as np
from PIL import Image
from paddleocr import PaddleOCR
import torch 
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, BitsAndBytesConfig



def model():

    bnb_config = BitsAndBytesConfig(
        llm_int8_enable_fp32_cpu_offload=True,
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    device_map = {
        "transformer.word_embeddings": 0,
        "transformer.word_embeddings_layernorm": 0,
        "lm_head": 0,
        "transformer.h": 0,
        "transformer.ln_f": 0,
        "model.embed_tokens": 0,
        "model.layers":0,
        "model.norm":0    
    }

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model_id="mychen76/mistral7b_ocr_to_json_v1"

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        quantization_config=bnb_config,
        # device_map=device_map
    )

    tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          trust_remote_code=True)


    return model, tokenizer





if __name__ == "__main__":

    receipt_texts = """ 
        The Lone Pine
        43 Manchester Road
        12480 Brisbane
        Australia
        617-3236-6207
        Invoice 08000008
        09/04/08
        Table
        25
        12:45
        2 Carlsberg Bottle
        16.00
        3 Heineken Draft Standard
        24.60
        1 Heineken Draft Half Liter
        15.20
        2 Carlsberg Bucket (5 bottles)
        80.00
        4 Grilled Chicken Breast
        74.00
        3 Sirloin Steak
        96.00
        1 Coke
        3.50
        5 Ice Cream
        18.00
        Subtotal
        327.30
        Sales/Gov Tax - 5%
        16.36
        Service Charge - 10%
        32.73
        GRAND TOTAL
        376.40
        Thank you and
        Cash
        400.00
        see you again!
        Change
        23.60
        John
        Bring this bill back within the next 10 days
        and get 15% discount on that day's food bill.
        """

    prompt=f"""### Instruction:
    You are POS receipt data expert, parse, detect, recognize and convert following receipt OCR image result into structure receipt data object. 
    Don't make up value not in the Input. Output must be a well-formed JSON object.```json

    ### Input:
    {receipt_texts}

    ### Output:
    """

    load_model, tokenizer = model()

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    with torch.inference_mode():
        inputs = tokenizer(prompt,
                        return_tensors="pt",
                        truncation=True).to(device)
        
       
        
        outputs = load_model.generate(**inputs, max_new_tokens=512)
        outputs = tokenizer.batch_decode(outputs)
        print(outputs)

        output_file = "receipt_output.txt"

        # Open a new file in write mode
        with open(output_file, "w") as f:
            # Write the raw output text to the file
            # The [0] is because tokenizer.batch_decode returns a list
            # of one string, so we want to get the first and only element.
            f.write(outputs[0])



```
    <s> ### Instruction:
    You are POS receipt data expert, parse, detect, recognize and convert following receipt OCR image result into structure receipt data object. 
    Don't make up value not in the Input. Output must be a well-formed JSON object.```json

    ### Input:
     
        The Lone Pine
        43 Manchester Road
        12480 Brisbane
        Australia
        617-3236-6207
        Invoice 08000008
        09/04/08
        Table
        25
        12:45
        2 Carlsberg Bottle
        16.00
        3 Heineken Draft Standard
        24.60
        1 Heineken Draft Half Liter
        15.20
        2 Carlsberg Bucket (5 bottles)
        80.00
        4 Grilled Chicken Breast
        74.00
        3 Sirloin Steak
        96.00
        1 Coke
        3.50
        5 Ice Cream
        18.00
        Subtotal
        327.30
        Sales/Gov Tax - 5%
        16.36
        Service Charge - 10%
        32.73
        GRAND TOTAL
        376.40
        Thank you and
        Cash
        400.00
        see you again!
        Change
        23.60
        John
        Bring this bill back within the next 10 days
        and get 15% discount on that day's food bill.
        

    ### Output:
    
        {
            "store": {
                "name": "The Lone Pine",
                "address": {
                    "street": "43 Manchester Road",
                    "city": "Brisbane",
                    "zip": "12480",
                    "country": "Australia"
                },
                "phone": "617-3236-6207"
            },
            "invoice": "08000008",
            "date": "09/04/08",
            "table": "25",
            "time": "12:45",
            "items": [
                {
                    "name": "Carlsberg Bottle",
                    "price": 16.00,
                    "quantity": 2
                },
                {
                    "name": "Heineken Draft Standard",
                    "price": 24.60,
                    "quantity": 3
                },
                {
                    "name": "Heineken Draft Half Liter",
                    "price": 15.20,
                    "quantity": 1
                },
                {
                    "name": "Carlsberg Bucket (5 bottles)",
                    "price": 80.00,
                    "quantity": 2
                },
                {
                    "name": "Grilled Chicken Breast",
                    "price": 74.00,
                    "quantity": 4
                },
                {
                    "name": "Sirloin Steak",
                    "price": 96.00,
                    "quantity": 3
                },
                {
                    "name": "Coke",
                    "price": 3.50,
                    "quantity": 1
                },
                {
                    "name": "Ice Cream",
                    "price": 18.00,
                    "quantity": 5
                }
```